In [ ]:
import tensorflow as tf
import yaml
import os
from object_detection.utils import dataset_util
import PIL
import numpy as np
from glob import glob
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
%matplotlib inline

In [ ]:
# Model evaluation on test images
test_image_dir = '/home/td/test_tl_detector/'
model_pb       = '/home/td/CarND-Capstone/ros/src/tl_detector/light_classification/models_frozen/frozen_srb_simon_tf1-3.pb'
#model_pb       = '/home/td/CarND-Capstone/ros/src/tl_detector/light_classification/models_frozen/frozen_real_tf1-3.pb'
#model_pb       = '/home/td/CarND-Capstone/ros/src/tl_detector/light_classification/models_frozen/frozen_sim_tf1-3.pb'
label_path     = '/home/td/test_tl_detector/label_map.pbtxt'

In [ ]:
# Helper function for batch evaluation
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
import time 

%matplotlib inline
NUM_CLASSES = 4

def batch_evaluate(model_pb, label_path, test_image_dir, output_dir=None):
    
    label_map = label_map_util.load_labelmap(label_path)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    TEST_IMAGE_PATHS = glob(os.path.join(test_image_dir, '*.jpg'))
    detection_graph = tf.Graph()
    
    def load_image_into_numpy_array(image):
        (im_width, im_height) = image.size
        return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
    
    with detection_graph.as_default():
        
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(model_pb, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    
        with tf.Session(graph=detection_graph) as sess:
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            for image_path in TEST_IMAGE_PATHS:
                image = PIL.Image.open(image_path)
                # the array based representation of the image will be used later in order to prepare the
                # result image with boxes and labels on it.
                image_np = load_image_into_numpy_array(image)
                # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
                image_np_expanded = np.expand_dims(image_np, axis=0)
    
                time0 = time.time()
    
                # Actual detection.
                (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: image_np_expanded})
    
                time1 = time.time()
    
                boxes = np.squeeze(boxes)
                scores = np.squeeze(scores)
                classes = np.squeeze(classes).astype(np.int32)
                
                # Visualization of the results of a detection.
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_np, boxes, classes, scores,
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=6)
                
                if output_dir is None:
                    plt.figure(figsize=(12, 8))
                    plt.imshow(image_np)
                    plt.show()
                else:
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    
                    output_path = os.path.split(image_path)[1]
                    output_path = os.path.splitext(output_path)[0] + '.png'
                    output_path = os.path.join(output_dir, output_path)
                    plt.imsave(output_path, image_np)


In [ ]:
# Evaluate trained model on test images
output_dir = os.path.join(test_image_dir, 'ssdv2_srb')
#output_dir = os.path.join(test_image_dir, 'ssdv2_real')
#output_dir = os.path.join(test_image_dir, 'ssdv2_sim')
batch_evaluate(model_pb, label_path, test_image_dir, output_dir)